In [1]:
import pandas as pd
import psycopg2 

In [2]:
conn = psycopg2.connect(
    host = "localhost",
    dbname = "interview_problems",
    user = "postgres",
    password = "22102003Goga",
    port = '5432')

# 1. Second Highest Salary

In [3]:
sql = "SELECT * FROM employee"
data = pd.read_sql(sql, conn)

data

,id,salary
0,1,1000
1,2,2000
2,3,3000
3,4,2000


<b>Solution A</b>

In [4]:
sql_1 = """
    SELECT MAX(salary) AS SecondHighestSalary
    FROM Employee
    WHERE salary != (SELECT MAX(salary) FROM Employee)
"""

data_1 = pd.read_sql(sql_1, conn)

data_1

,secondhighestsalary
0,2000


<b>Solution B</b>

In [5]:
sql_2 = """
    SELECT
        COALESCE(
            (SELECT DISTINCT Salary
            FROM Employee
            ORDER BY Salary DESC
            LIMIT 1 OFFSET 1
            ), null) as SecondHighestSalary
    FROM Employee
    LIMIT 1
"""

data_2 = pd.read_sql(sql_2, conn)

data_2

,secondhighestsalary
0,2000


# 2. Duplicate Emails

In [6]:
sql = "SELECT * FROM emails"
data = pd.read_sql(sql, conn)

data

,id,email
0,1,abc@.com
1,2,acb@.com
2,3,abc@.com


<b>Solution A</b>

In [7]:
sql_1 = """
    SELECT Email
    FROM (
        SELECT Email, count(Email) AS count
        FROM Emails
        GROUP BY Email
    ) as email_count
    WHERE count > 1
"""

data_1 = pd.read_sql(sql_1, conn)

data_1

,email
0,abc@.com


<b>Solution B</b>

In [8]:
sql_2 = """
    SELECT Email
    FROM Emails
    GROUP BY Email
    HAVING count(Email) > 1
"""

data_2 = pd.read_sql(sql_2, conn)

data_2

,email
0,abc@.com


# 3. Rising Temperature

In [9]:
sql = "SELECT * FROM weather"
data = pd.read_sql(sql, conn)

data

,id,date,temperature
0,1,2015-01-01,10
1,2,2015-01-02,20
2,3,2015-01-03,15
3,4,2015-01-04,30


<b>Solution A</b>

In [10]:
sql_1 = """
    SELECT DISTINCT a.Id
    FROM Weather a, Weather b
    WHERE a.Temperature > b.Temperature
    AND (a.date - b.date) = 1
"""

data_1 = pd.read_sql(sql_1, conn)

data_1

,id
0,2
1,4


# 4. Department Highest Salary

In [11]:
sql_employee = "SELECT * FROM employee_for_department"
sql_department = "SELECT * FROM department"

data_employee = pd.read_sql(sql_employee, conn)
data_department = pd.read_sql(sql_department, conn)

In [12]:
data_employee

,id,name,salary,departmentid
0,1,Ihor,10000,1
1,2,Max,5000,1
2,3,Joe,7000,2
3,4,Katya,3000,2
4,5,Anna,8000,1


In [13]:
data_department

,id,name
0,1,Sales
1,2,IT


<b>Solution A</b>

In [14]:
sql_1 = """
    SELECT
        Department.name AS Department,
        employee_for_department.name AS Employee,
        Salary
    FROM employee_for_department
    INNER JOIN Department ON employee_for_department.DepartmentId = Department.Id
    WHERE (DepartmentId , Salary) 
        IN
        (   SELECT
                DepartmentId, MAX(Salary)
            FROM
                employee_for_department
            GROUP BY DepartmentId
     )
"""

data_1 = pd.read_sql(sql_1, conn)

data_1

,department,employee,salary
0,Sales,Ihor,10000
1,IT,Joe,7000
